## Preprocesing

In [5]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [7]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_page= requests.get(wikipedia_link).text
soup = BeautifulSoup(raw_wikipedia_page,'xml')

### Extracting Raw Table from Web Page

In [9]:
table = soup.find('table')
Postcode      = []
Borough       = []
Neighbourhood = []
for tr_cell in table.find_all('tr'):    
    counter = 1
    Postcode_var      = -1
    Borough_var       = -1
    Neighbourhood_var = -1    
    for td_cell in tr_cell.find_all('td'):
        if counter == 1: 
            Postcode_var = td_cell.text
        if counter == 2: 
            Borough_var = td_cell.text
            tag_a_Borough = td_cell.find('a')            
        if counter == 3: 
            Neighbourhood_var = str(td_cell.text).strip()
            tag_a_Neighbourhood = td_cell.find('a')            
        counter += 1        
        if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned' or Neighbourhood_var == 'Not assigned'):
            continue
    try:
        if ((tag_a_Borough is None) or (tag_a_Neighbourhood is None)):
            continue
    except:
        pass    
    if(Postcode_var == -1 or Borough_var == -1 or Neighbourhood_var == -1):
        continue    
    Postcode.append(Postcode_var)
    Borough.append(Borough_var)
    Neighbourhood.append(Neighbourhood_var)

### Integrating Postal codes with more than 1 neighbour

In [10]:
unique_p = set(Postcode)
print('num of unique Postal codes:', len(unique_p))
Postcode_u      = []
Borough_u       = []
Neighbourhood_u = []
for postcode_unique_element in unique_p:
    p_var = ''; b_var = ''; n_var = ''; 
    for postcode_idx, postcode_element in enumerate(Postcode):
        if postcode_unique_element == postcode_element:
            p_var = postcode_element;
            b_var = Borough[postcode_idx]
            if n_var == '': 
                n_var = Neighbourhood[postcode_idx]
            else:
                n_var = n_var + ', ' + Neighbourhood[postcode_idx]
    Postcode_u.append(p_var)
    Borough_u.append(b_var)
    Neighbourhood_u.append(n_var)

num of unique Postal codes: 78


### Creating the Dataframe

In [13]:
toronto_dict = {'Postcode':Postcode_u, 'Borough':Borough_u, 'Neighbourhood':Neighbourhood_u}
df = pd.DataFrame.from_dict(toronto_dict)
df.head()

,Postcode,Borough,Neighbourhood
0,M5A,Downtown Toronto,Harbourfront
1,M5S,Downtown Toronto,University of Toronto
2,M4E,East Toronto,The Beaches
3,M4V,Central Toronto,"Deer Park, Rathnelly, South Hill"
4,M5C,Downtown Toronto,St. James Town


In [14]:
df.shape

(78, 3)